In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modified-hrmodel/modified_model.h5
/kaggle/input/calssification-phoneme-model/phoneme_classification_model.h5
/kaggle/input/a-z-combineddataset/a_z_combibed.csv
/kaggle/input/huggingface-captone/capstone_dataset_hugging_face.csv


**accuracy : 83%***   using lstm

In [ ]:
# accuracy : 90%  bilstm

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-09-26 02:58:59.806307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 02:58:59.806426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 02:58:59.939902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load the dataset
df = pd.read_csv('/kaggle/input/huggingface-captone/capstone_dataset_hugging_face.csv')
# Create labels and concatenate phoneme sequences
X = pd.concat([df['ipa_phoneme'], df['incorrect_ipa_phoneme']], axis=0)
y = pd.concat([pd.Series([0] * len(df)), pd.Series([1] * len(df))], axis=0)

# Tokenize the phonemes
tokenizer = Tokenizer(char_level=True)  # character-level tokenization
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, padding='post')

In [24]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, X_padded.shape[1]))  # Ensure the model is built with the correct input shape
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Save the model
# model.save('phoneme_classification_model.h5')
model.save_weights('/kaggle/working/save/weights')


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 34, 64)         │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,697 (303.50 KB)

 Trainable params: 77,697 (303.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 106s 7ms/step - accuracy: 0.7440 - loss: 0.4878 - val_accuracy: 0.8626 - val_loss: 0.3238
Epoch 2/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.8686 - loss: 0.3124 - val_accuracy: 0.8788 - val_loss: 0.2925
Epoch 3/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.8829 - loss: 0.2863 - val_accuracy: 0.8860 - val_loss: 0.2804
Epoch 4/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.8899 - loss: 0.2724 - val_accuracy: 0.8912 - val_loss: 0.2725
Epoch 5/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.8944 - loss: 0.2632 - val_accuracy: 0.8931 - val_loss: 0.2683
Epoch 6/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.8977 - loss: 0.2556 - val_accuracy: 0.8957 - val_loss: 0.2628
Epoch 7/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - accuracy: 0.9016 - loss: 0.2485 - val_accuracy: 0.8961 - val_loss: 0.2606
Epoch 8/50
13747/13747 ━━━━━━━━━━━━━━━━━━━━ 102s 7ms/step - ac

ValueError: The filename must end in `.weights.h5`. Received: filepath=/kaggle/working/save/weights

In [5]:
import tensorflow as tf

In [6]:
model.save('phoneme_classification_model.h5')

In [3]:
# Save the model

loaded_model= tf.keras.models.load_model('/kaggle/input/calssification-phoneme-model/phoneme_classification_model.h5')


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Assuming you have the original dataset loaded as df
X = pd.concat([df['ipa_phoneme'], df['incorrect_ipa_phoneme']], axis=0)

# Recreate the tokenizer
tokenizer = Tokenizer(char_level=True)  # character-level tokenization
tokenizer.fit_on_texts(X)


In [20]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a function to preprocess and predict for each phoneme
def predict_each_phoneme(loaded_model, tokenizer, phonemes):
    predictions = []
    for phoneme in phonemes:
        # Tokenize the phoneme
        phoneme_tokenized = tokenizer.texts_to_sequences([phoneme])
        phoneme_padded = pad_sequences(phoneme_tokenized, padding='post', maxlen=X_padded.shape[1])

        # Predict using the model
        prediction = loaded_model.predict(phoneme_padded)

        # Interpret the prediction
        if prediction[0] > 0.5:
            predictions.append((phoneme, "Mispronounced"))
        else:
            predictions.append((phoneme, "Correctly pronounced"))
    
    return predictions

# Recreate the tokenizer using the original dataset
X = pd.concat([df['ipa_phoneme'], df['incorrect_ipa_phoneme']], axis=0)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(X)

# Load the trained model
# loaded_model = tf.keras.models.load_model('/path/to/saved/model')

# Example phonemes list (assuming input as a list of phonemes)
# phonemes_list =['ɐ pˈɔːkɐlˌɒpsiː','ɪz','kˈʌmɪŋ','ˈɔːlweɪz','juː','wˈɒnt','tuː',' sˈiː','aɪtˈiː','ɪnðə','suːpˈɜːlətˌɪv','dɪɡɹˈiː']  # Replace with actual phoneme sequence
phonemes_list =["kˈælkjʊlˌeɪɾɚ"] 
# Predict for each phoneme
results = predict_each_phoneme(loaded_model, tokenizer, phonemes_list)

# Print the results
for phoneme, result in results:
    print(f'Phoneme: {phoneme}, Result: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Phoneme: kˈælkjʊlˌeɪɾɚ, Result: Correctly pronounced
